In [1]:
import pandas as pd


In [2]:
raw_data = pd.read_csv('data/judge-1377884607_tweet_product_company.csv', encoding = 'latin1')
raw_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9093 entries, 0 to 9092
Data columns (total 3 columns):
tweet_text                                            9092 non-null object
emotion_in_tweet_is_directed_at                       3291 non-null object
is_there_an_emotion_directed_at_a_brand_or_product    9093 non-null object
dtypes: object(3)
memory usage: 213.2+ KB


In [15]:
raw_data.head(10)
#found the null - drop row 6

,tweet_text,emotion_in_tweet_is_directed_at,is_there_an_emotion_directed_at_a_brand_or_product
0,.@wesley83 I have a 3G iPhone. After 3 hrs twe...,iPhone,Negative emotion
1,@jessedee Know about @fludapp ? Awesome iPad/i...,iPad or iPhone App,Positive emotion
2,@swonderlin Can not wait for #iPad 2 also. The...,iPad,Positive emotion
3,@sxsw I hope this year's festival isn't as cra...,iPad or iPhone App,Negative emotion
4,@sxtxstate great stuff on Fri #SXSW: Marissa M...,Google,Positive emotion
5,@teachntech00 New iPad Apps For #SpeechTherapy...,NaN,No emotion toward brand or product
6,NaN,NaN,No emotion toward brand or product
7,"#SXSW is just starting, #CTIA is around the co...",Android,Positive emotion
8,Beautifully smart and simple idea RT @madebyma...,iPad or iPhone App,Positive emotion
9,Counting down the days to #sxsw plus strong Ca...,Apple,Positive emotion


## Data Exploration and Cleaning

In [4]:
raw_data['emotion_in_tweet_is_directed_at'].value_counts()

#sort these into google or apple?

iPad                               946
Apple                              661
iPad or iPhone App                 470
Google                             430
iPhone                             297
Other Google product or service    293
Android App                         81
Android                             78
Other Apple product or service      35
Name: emotion_in_tweet_is_directed_at, dtype: int64

In [16]:
raw_data['is_there_an_emotion_directed_at_a_brand_or_product'].value_counts()

# 3 classes

No emotion toward brand or product    5389
Positive emotion                      2978
Negative emotion                       570
I can't tell                           156
Name: is_there_an_emotion_directed_at_a_brand_or_product, dtype: int64

So there's tweets not in english, some tweets that look actually neutral, and some like the one I've selected below maybe are sarcastic but even I, a human, can't tell. It seems unrealistic to go through all of these and it also doesn't make sense to have an 'idk' class so I'm going to relabel them as neutral along with the no emotion tweets.

In [25]:
ct_df = raw_data[raw_data['is_there_an_emotion_directed_at_a_brand_or_product'] == 'I can\'t tell']
ct_df.head(20)


,tweet_text,emotion_in_tweet_is_directed_at,is_there_an_emotion_directed_at_a_brand_or_product
90,Thanks to @mention for publishing the news of ...,NaN,I can't tell
102,ÛÏ@mention &quot;Apple has opened a pop-up st...,NaN,I can't tell
237,Just what America needs. RT @mention Google to...,NaN,I can't tell
341,The queue at the Apple Store in Austin is FOUR...,NaN,I can't tell
368,Hope it's better than wave RT @mention Buzz is...,NaN,I can't tell
413,SYD #SXSW crew your iPhone extra juice pods ha...,NaN,I can't tell
441,Why Barry Diller thinks iPad only content is n...,NaN,I can't tell
488,Gave into extreme temptation at #SXSW and boug...,NaN,I can't tell
640,Catch 22Û_ I mean iPad 2 at #SXSW : {link},NaN,I can't tell
705,Forgot my iPhone for #sxsw. Android only. Knif...,NaN,I can't tell


In [31]:
ct_df.loc[237][0]

'Just what America needs. RT @mention Google to Launch Major New Social Network Called Circles, Possibly Today {link} #sxsw'

In [ ]:
Now, taking a look at those emotionless tweets, 

In [24]:
ne_df = raw_data[raw_data['is_there_an_emotion_directed_at_a_brand_or_product'] == 'No emotion toward brand or product']

Finally, we're missing a lot of labels. When I look at the nulls, I can actually see a lot of the product mentions in them...

In [12]:
# lots of products are null
nulls_df = raw_data[raw_data['emotion_in_tweet_is_directed_at'].isna()]

In [13]:
len(nulls_df)

5802

In [14]:
nulls_df.head()

,tweet_text,emotion_in_tweet_is_directed_at,is_there_an_emotion_directed_at_a_brand_or_product
5,@teachntech00 New iPad Apps For #SpeechTherapy...,NaN,No emotion toward brand or product
6,NaN,NaN,No emotion toward brand or product
16,Holler Gram for iPad on the iTunes App Store -...,NaN,No emotion toward brand or product
32,"Attn: All #SXSW frineds, @mention Register fo...",NaN,No emotion toward brand or product
33,Anyone at #sxsw want to sell their old iPad?,NaN,No emotion toward brand or product


In [ ]:
nulls_df.head()